# Day 14 - claude

In [2]:
class Robot:
    def __init__(self, pos_x, pos_y, vel_x, vel_y):
        self.pos_x = pos_x
        self.pos_y = pos_y
        self.vel_x = vel_x
        self.vel_y = vel_y

def parse_input(filename):
    robots = []
    with open(filename, 'r') as f:
        for line in f:
            # Split into position and velocity parts
            pos, vel = line.strip().split(' ')
            
            # Parse position
            pos = pos[2:]  # Remove 'p='
            pos_x, pos_y = map(int, pos.split(','))
            
            # Parse velocity
            vel = vel[2:]  # Remove 'v='
            vel_x, vel_y = map(int, vel.split(','))
            
            robots.append(Robot(pos_x, pos_y, vel_x, vel_y))
    return robots

def simulate_movement(robots, width, height, seconds):
    # For each robot
    for robot in robots:
        # Calculate final position after wrapping
        robot.pos_x = (robot.pos_x + robot.vel_x * seconds) % width
        robot.pos_y = (robot.pos_y + robot.vel_y * seconds) % height

def count_robots_in_quadrants(robots, width, height):
    # Initialize counters for each quadrant
    quadrants = [0] * 4
    
    # Define midpoints
    mid_x = width // 2
    mid_y = height // 2
    
    for robot in robots:
        # Skip robots on the middle lines
        if robot.pos_x == mid_x or robot.pos_y == mid_y:
            continue
        
        # Determine quadrant (0-3, starting from top-left, going clockwise)
        if robot.pos_x < mid_x:
            if robot.pos_y < mid_y:
                quadrants[0] += 1  # Top-left
            else:
                quadrants[3] += 1  # Bottom-left
        else:
            if robot.pos_y < mid_y:
                quadrants[1] += 1  # Top-right
            else:
                quadrants[2] += 1  # Bottom-right
    
    return quadrants

def calculate_safety_factor(quadrants):
    return quadrants[0] * quadrants[1] * quadrants[2] * quadrants[3]

def main():
    # Constants
    WIDTH = 101
    HEIGHT = 103
    SECONDS = 100
    
    # Parse input and get robots
    robots = parse_input('input.txt')
    
    # Simulate movement
    simulate_movement(robots, WIDTH, HEIGHT, SECONDS)
    
    # Count robots in quadrants
    quadrants = count_robots_in_quadrants(robots, WIDTH, HEIGHT)
    
    # Calculate and print safety factor
    safety_factor = calculate_safety_factor(quadrants)
    print(f"Safety factor after {SECONDS} seconds: {safety_factor}")
    print(f"Robots in quadrants (TL, TR, BR, BL): {quadrants}")

if __name__ == "__main__":
    main()

Safety factor after 100 seconds: 229421808
Robots in quadrants (TL, TR, BR, BL): [132, 133, 108, 121]


## Part 2

In [3]:
class Robot:
    def __init__(self, pos_x, pos_y, vel_x, vel_y):
        self.pos_x = pos_x
        self.pos_y = pos_y
        self.vel_x = vel_x
        self.vel_y = vel_y

def parse_input(filename):
    robots = []
    with open(filename, 'r') as f:
        for line in f:
            # Split into position and velocity parts
            pos, vel = line.strip().split(' ')
            
            # Parse position
            pos = pos[2:]  # Remove 'p='
            pos_x, pos_y = map(int, pos.split(','))
            
            # Parse velocity
            vel = vel[2:]  # Remove 'v='
            vel_x, vel_y = map(int, vel.split(','))
            
            robots.append(Robot(pos_x, pos_y, vel_x, vel_y))
    return robots

def simulate_step(robots, width, height):
    # Move each robot one step
    for robot in robots:
        robot.pos_x = (robot.pos_x + robot.vel_x) % width
        robot.pos_y = (robot.pos_y + robot.vel_y) % height

def get_robot_positions(robots):
    # Return set of (x,y) positions where robots are located
    return {(robot.pos_x, robot.pos_y) for robot in robots}

def is_christmas_tree(positions, width, height):
    # Convert positions to a grid for easier pattern matching
    grid = [[0] * width for _ in range(height)]
    for x, y in positions:
        grid[y][x] = 1
    
    # Find center of mass to locate potential tree
    total_x = sum(x for x, _ in positions)
    total_y = sum(y for _, y in positions)
    center_x = total_x // len(positions)
    center_y = total_y // len(positions)
    
    # Check for tree-like characteristics
    # 1. Should have a vertical trunk
    trunk_height = 0
    trunk_x = center_x
    trunk_y = center_y
    
    while trunk_y < height and grid[trunk_y][trunk_x] == 1:
        trunk_height += 1
        trunk_y += 1
    
    if trunk_height < 3:  # Minimum trunk height
        return False
    
    # 2. Should have triangular branching pattern above trunk
    branches_found = 0
    max_width = 1
    y = center_y - 1  # Start above trunk
    
    while y >= 0 and y > center_y - 5:  # Look for up to 4 layers of branches
        row_robots = sum(grid[y][center_x-max_width:center_x+max_width+1])
        if row_robots > max_width:
            branches_found += 1
        max_width += 1
        y -= 1
    
    # Need at least 3 good branch layers
    return branches_found >= 3

def find_christmas_tree(robots, width, height, max_seconds=1000):
    original_positions = [(r.pos_x, r.pos_y) for r in robots]
    
    for second in range(max_seconds):
        if second > 0:  # Don't simulate first step
            simulate_step(robots, width, height)
        
        positions = get_robot_positions(robots)
        
        # Check if current arrangement looks like a Christmas tree
        if is_christmas_tree(positions, width, height):
            return second
        
        # Check if we've returned to original positions (cycle found)
        current_positions = [(r.pos_x, r.pos_y) for r in robots]
        if current_positions == original_positions and second > 0:
            return None  # No tree pattern found within a full cycle
    
    return None  # No tree pattern found within max_seconds

def main():
    # Constants
    WIDTH = 101
    HEIGHT = 103
    
    # Parse input and get robots
    robots = parse_input('input.txt')
    
    # Find the first occurrence of a Christmas tree pattern
    seconds = find_christmas_tree(robots, WIDTH, HEIGHT)
    
    if seconds is not None:
        print(f"Christmas tree pattern found after {seconds} seconds!")
    else:
        print("No Christmas tree pattern found within the time limit.")

if __name__ == "__main__":
    main()

No Christmas tree pattern found within the time limit.


In [5]:
class Robot:
    def __init__(self, pos_x, pos_y, vel_x, vel_y):
        self.pos_x = pos_x
        self.pos_y = pos_y
        self.vel_x = vel_x
        self.vel_y = vel_y

def parse_input(filename):
    robots = []
    with open(filename, 'r') as f:
        for line in f:
            # Split into position and velocity parts
            pos, vel = line.strip().split(' ')
            
            # Parse position
            pos = pos[2:]  # Remove 'p='
            pos_x, pos_y = map(int, pos.split(','))
            
            # Parse velocity
            vel = vel[2:]  # Remove 'v='
            vel_x, vel_y = map(int, vel.split(','))
            
            robots.append(Robot(pos_x, pos_y, vel_x, vel_y))
    return robots

def simulate_step(robots, width, height):
    # Move each robot one step
    for robot in robots:
        robot.pos_x = (robot.pos_x + robot.vel_x) % width
        robot.pos_y = (robot.pos_y + robot.vel_y) % height

def get_grid_snapshot(robots, width, height):
    # Create a grid representation of robot positions
    grid = [['.'] * width for _ in range(height)]
    for robot in robots:
        grid[robot.pos_y][robot.pos_x] = '#'
    return grid

def find_message(robots, width, height, max_seconds=10000):
    seen_states = set()
    
    def get_state_hash(robots):
        # Create a hashable representation of robot positions
        return tuple(sorted((r.pos_x, r.pos_y) for r in robots))
    
    for second in range(max_seconds):
        # Get current state
        current_state = get_state_hash(robots)
        
        # If we've seen this state before, we're in a cycle
        if current_state in seen_states:
            print(f"Cycle detected at {second} seconds")
            return None
        
        # Add current state to seen states
        seen_states.add(current_state)
        
        # Get current grid
        grid = get_grid_snapshot(robots, width, height)
        
        # Check if robots form any readable message
        # Print the current state every 100 steps or when robots are close together
        if second % 100 == 0:
            print(f"\nTime: {second} seconds")
            robot_positions = [(r.pos_x, r.pos_y) for r in robots]
            x_coords, y_coords = zip(*robot_positions)
            spread = max(max(x_coords) - min(x_coords), max(y_coords) - min(y_coords))
            if spread < 20:  # If robots are close together
                print_grid_section(grid, robot_positions)
        
        # Move robots
        simulate_step(robots, width, height)
    
    return None

def print_grid_section(grid, positions):
    # Find the bounds of robot positions
    x_coords, y_coords = zip(*positions)
    min_x, max_x = max(0, min(x_coords) - 2), min(len(grid[0]), max(x_coords) + 3)
    min_y, max_y = max(0, min(y_coords) - 2), min(len(grid), max(y_coords) + 3)
    
    # Print the relevant section of the grid
    for y in range(min_y, max_y):
        row = ''.join(grid[y][min_x:max_x])
        print(row)
    print()

def main():
    # Constants
    WIDTH = 101
    HEIGHT = 103
    
    # Parse input and get robots
    robots = parse_input('input.txt')
    
    # Find when robots form a message
    seconds = find_message(robots, WIDTH, HEIGHT)
    
    if seconds is not None:
        print(f"Message found after {seconds} seconds!")
    else:
        print("No clear message found within the time limit.")

if __name__ == "__main__":
    main()


Time: 0 seconds

Time: 100 seconds

Time: 200 seconds

Time: 300 seconds

Time: 400 seconds

Time: 500 seconds

Time: 600 seconds

Time: 700 seconds

Time: 800 seconds

Time: 900 seconds

Time: 1000 seconds

Time: 1100 seconds

Time: 1200 seconds

Time: 1300 seconds

Time: 1400 seconds

Time: 1500 seconds

Time: 1600 seconds

Time: 1700 seconds

Time: 1800 seconds

Time: 1900 seconds

Time: 2000 seconds

Time: 2100 seconds

Time: 2200 seconds

Time: 2300 seconds

Time: 2400 seconds

Time: 2500 seconds

Time: 2600 seconds

Time: 2700 seconds

Time: 2800 seconds

Time: 2900 seconds

Time: 3000 seconds

Time: 3100 seconds

Time: 3200 seconds

Time: 3300 seconds

Time: 3400 seconds

Time: 3500 seconds

Time: 3600 seconds

Time: 3700 seconds

Time: 3800 seconds

Time: 3900 seconds

Time: 4000 seconds

Time: 4100 seconds

Time: 4200 seconds

Time: 4300 seconds

Time: 4400 seconds

Time: 4500 seconds

Time: 4600 seconds

Time: 4700 seconds

Time: 4800 seconds

Time: 4900 seconds

Time: 5000 

In [6]:
from math import gcd
from functools import reduce

class Robot:
    def __init__(self, pos_x, pos_y, vel_x, vel_y):
        self.pos_x = pos_x
        self.pos_y = pos_y
        self.vel_x = vel_x
        self.vel_y = vel_y

def parse_input(filename):
    robots = []
    with open(filename, 'r') as f:
        for line in f:
            pos, vel = line.strip().split(' ')
            pos = pos[2:]  # Remove 'p='
            pos_x, pos_y = map(int, pos.split(','))
            vel = vel[2:]  # Remove 'v='
            vel_x, vel_y = map(int, vel.split(','))
            robots.append(Robot(pos_x, pos_y, vel_x, vel_y))
    return robots

def lcm(a, b):
    return abs(a * b) // gcd(a, b)

def lcm_list(numbers):
    return reduce(lcm, numbers)

def get_alignment_time(robots, width, height):
    # For each robot, calculate when it aligns in x and y directions
    x_cycles = []
    y_cycles = []
    
    for robot in robots:
        if robot.vel_x != 0:
            # Time to return to starting x position
            x_period = width // gcd(abs(robot.vel_x), width)
            x_cycles.append(x_period)
        
        if robot.vel_y != 0:
            # Time to return to starting y position
            y_period = height // gcd(abs(robot.vel_y), height)
            y_cycles.append(y_period)
    
    # Find the least common multiple of all periods
    # This is when robots will align in both dimensions
    x_alignment = lcm_list(x_cycles) if x_cycles else 1
    y_alignment = lcm_list(y_cycles) if y_cycles else 1
    
    return lcm(x_alignment, y_alignment)

def simulate_and_check(robots, width, height, target_time):
    # Create a copy of robots to simulate
    simulated_robots = [Robot(r.pos_x, r.pos_y, r.vel_x, r.vel_y) for r in robots]
    
    # Move each robot to its position at target_time
    for robot in simulated_robots:
        robot.pos_x = (robot.pos_x + robot.vel_x * target_time) % width
        robot.pos_y = (robot.pos_y + robot.vel_y * target_time) % height
    
    # Return the positions for visualization
    return [(r.pos_x, r.pos_y) for r in simulated_robots]

def print_grid(positions, width, height):
    grid = [['.'] * width for _ in range(height)]
    for x, y in positions:
        grid[y][x] = '#'
    
    # Print only the relevant section
    x_coords, y_coords = zip(*positions)
    min_x, max_x = max(0, min(x_coords) - 2), min(width, max(x_coords) + 3)
    min_y, max_y = max(0, min(y_coords) - 2), min(height, max(y_coords) + 3)
    
    print(f"\nTime: {target_time}")
    for y in range(min_y, max_y):
        print(''.join(grid[y][min_x:max_x]))

def main():
    WIDTH = 101
    HEIGHT = 103
    
    robots = parse_input('input.txt')
    
    # Calculate potential alignment time
    target_time = get_alignment_time(robots, WIDTH, HEIGHT)
    print(f"Checking alignment at time: {target_time}")
    
    # Simulate and check positions at target time
    positions = simulate_and_check(robots, WIDTH, HEIGHT, target_time)
    print_grid(positions, WIDTH, HEIGHT)

if __name__ == "__main__":
    main()

Checking alignment at time: 10403


NameError: name 'target_time' is not defined

In [7]:
from math import gcd
from functools import reduce

class Robot:
    def __init__(self, pos_x, pos_y, vel_x, vel_y):
        self.pos_x = pos_x
        self.pos_y = pos_y
        self.vel_x = vel_x
        self.vel_y = vel_y

def parse_input(filename):
    robots = []
    with open(filename, 'r') as f:
        for line in f:
            pos, vel = line.strip().split(' ')
            pos = pos[2:]  # Remove 'p='
            pos_x, pos_y = map(int, pos.split(','))
            vel = vel[2:]  # Remove 'v='
            vel_x, vel_y = map(int, vel.split(','))
            robots.append(Robot(pos_x, pos_y, vel_x, vel_y))
    return robots

def lcm(a, b):
    return abs(a * b) // gcd(a, b)

def lcm_list(numbers):
    return reduce(lcm, numbers)

def get_alignment_time(robots, width, height):
    # For each robot, calculate when it aligns in x and y directions
    x_cycles = []
    y_cycles = []
    
    for robot in robots:
        if robot.vel_x != 0:
            # Time to return to starting x position
            x_period = width // gcd(abs(robot.vel_x), width)
            x_cycles.append(x_period)
        
        if robot.vel_y != 0:
            # Time to return to starting y position
            y_period = height // gcd(abs(robot.vel_y), height)
            y_cycles.append(y_period)
    
    # Find the least common multiple of all periods
    x_alignment = lcm_list(x_cycles) if x_cycles else 1
    y_alignment = lcm_list(y_cycles) if y_cycles else 1
    
    return lcm(x_alignment, y_alignment)

def simulate_and_check(robots, width, height, time):
    # Create a copy of robots to simulate
    simulated_robots = [Robot(r.pos_x, r.pos_y, r.vel_x, r.vel_y) for r in robots]
    
    # Move each robot to its position at target_time
    for robot in simulated_robots:
        robot.pos_x = (robot.pos_x + robot.vel_x * time) % width
        robot.pos_y = (robot.pos_y + robot.vel_y * time) % height
    
    return simulated_robots

def check_nearby_times(robots, width, height, target_time, range_to_check=100):
    """Check a range of times around the target time for the pattern."""
    start_time = max(0, target_time - range_to_check)
    end_time = target_time + range_to_check
    
    for time in range(start_time, end_time + 1):
        simulated_robots = simulate_and_check(robots, width, height, time)
        positions = [(r.pos_x, r.pos_y) for r in simulated_robots]
        
        # Get the spread of robots
        x_coords, y_coords = zip(*positions)
        x_spread = max(x_coords) - min(x_coords)
        y_spread = max(y_coords) - min(y_coords)
        
        # If robots are close together, print the pattern
        if x_spread < 20 and y_spread < 20:
            print(f"\nChecking time: {time}")
            print_grid(positions, width, height, time)

def print_grid(positions, width, height, time):
    grid = [['.'] * width for _ in range(height)]
    for x, y in positions:
        grid[y][x] = '#'
    
    # Print only the relevant section
    x_coords, y_coords = zip(*positions)
    min_x = max(0, min(x_coords) - 2)
    max_x = min(width, max(x_coords) + 3)
    min_y = max(0, min(y_coords) - 2)
    max_y = min(height, max(y_coords) + 3)
    
    print(f"Time: {time}")
    for y in range(min_y, max_y):
        print(''.join(grid[y][min_x:max_x]))

def main():
    WIDTH = 101
    HEIGHT = 103
    
    robots = parse_input('input.txt')
    
    # Calculate potential alignment time
    target_time = get_alignment_time(robots, WIDTH, HEIGHT)
    print(f"Base alignment time calculated: {target_time}")
    
    # Check times around the calculated alignment time
    check_nearby_times(robots, WIDTH, HEIGHT, target_time)

if __name__ == "__main__":
    main()

Base alignment time calculated: 10403


In [10]:
from collections import defaultdict

class Robot:
    def __init__(self, pos_x, pos_y, vel_x, vel_y):
        self.pos_x = pos_x
        self.pos_y = pos_y
        self.vel_x = vel_x
        self.vel_y = vel_y
        
    def position_at_time(self, t, width, height):
        # Calculate position at given time t with wrapping
        x = (self.pos_x + self.vel_x * t) % width
        y = (self.pos_y + self.vel_y * t) % height
        return (x, y)

def parse_input(filename):
    robots = []
    with open(filename, 'r') as f:
        for line in f:
            pos, vel = line.strip().split(' ')
            pos = pos[2:]  # Remove 'p='
            pos_x, pos_y = map(int, pos.split(','))
            vel = vel[2:]  # Remove 'v='
            vel_x, vel_y = map(int, vel.split(','))
            robots.append(Robot(pos_x, pos_y, vel_x, vel_y))
    return robots

def get_positions_at_time(robots, t, width, height):
    # Returns a set of robot positions at time t
    positions = defaultdict(int)
    for robot in robots:
        pos = robot.position_at_time(t, width, height)
        positions[pos] += 1
    return positions

def is_christmas_tree(positions, width, height):
    # Define characteristics of a Christmas tree pattern:
    # - Should have a triangular shape
    # - Should have a trunk
    # - Should be roughly centered
    
    # Convert positions to a grid for easier pattern matching
    grid = [[0] * width for _ in range(height)]
    for (x, y), count in positions.items():
        grid[y][x] = count
    
    # Find the bounding box of non-zero cells
    min_x = width
    max_x = 0
    min_y = height
    max_y = 0
    
    for y in range(height):
        for x in range(width):
            if grid[y][x] > 0:
                min_x = min(min_x, x)
                max_x = max(max_x, x)
                min_y = min(min_y, y)
                max_y = max(max_y, y)
    
    if max_x - min_x < 5 or max_y - min_y < 7:
        return False  # Too small to be a tree
    
    # Check if roughly centered
    center_x = width // 2
    tree_center_x = (min_x + max_x) // 2
    if abs(center_x - tree_center_x) > width // 6:
        return False
    
    # Check for triangular shape (more robots at bottom than top)
    levels = [0] * (max_y - min_y + 1)
    for y in range(min_y, max_y + 1):
        for x in range(min_x, max_x + 1):
            levels[y - min_y] += grid[y][x]
    
    # Check if generally increasing towards middle
    mid_point = len(levels) // 2
    for i in range(1, mid_point):
        if levels[i] < levels[i-1]:
            return False
    
    # Check for trunk (concentration of robots at bottom)
    trunk_height = (max_y - min_y) // 4
    trunk_width = (max_x - min_x) // 3
    trunk_center_x = (min_x + max_x) // 2
    
    has_trunk = False
    for y in range(max_y - trunk_height, max_y + 1):
        trunk_count = sum(grid[y][x] for x in range(trunk_center_x - trunk_width//2, 
                                                   trunk_center_x + trunk_width//2 + 1))
        if trunk_count > 0:
            has_trunk = True
            break
    
    return has_trunk

def find_christmas_tree(robots, width, height, max_time):
    for t in range(max_time):
        positions = get_positions_at_time(robots, t, width, height)
        if is_christmas_tree(positions, width, height):
            return t
    return None

def main():
    WIDTH = 101
    HEIGHT = 103
    MAX_TIME = 1000  # Adjust as needed
    
    robots = parse_input('input.txt')
    result = find_christmas_tree(robots, WIDTH, HEIGHT, MAX_TIME)
    
    if result is not None:
        print(f"Christmas tree pattern found at t={result} seconds")
    else:
        print(f"No Christmas tree pattern found within {MAX_TIME} seconds")

if __name__ == "__main__":
    main()

No Christmas tree pattern found within 1000 seconds
